In [ ]:
import ee, h5py, warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

###
# ------- Set up local proxy to access GEE, may not required for all people ------- #
###
    
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10793)
socket.socket = socks.socksocket

In [ ]:
def regroup_365days(raw_data, min_prec = 0):

    raw_data2 = raw_data + 0.0
    # raw_data2['PREC'] = raw_data2['RAIN'] + raw_data2['SNOW']

    # raw_data2.index = raw_data2.index.set_names('time') #.to_xarray()#.convert_calendar('noleap')
    # raw_data2 = raw_data2.to_xarray().convert_calendar('noleap')

    raw_data_group = raw_data2.groupby(raw_data2.index.dayofyear).mean()#.to_dataframe()
        
    raw_data_group['PRECC'][raw_data_group['PRECC'] <= min_prec ] = 0 # fix precipitation < 0.1 mm/day
 
    raw_data_group.index = pd.date_range(raw_data.index[0], periods=len(raw_data_group.PRECC.values))

    return raw_data_group


In [ ]:
def ee_array_to_df(arr, list_of_bands):

    # Transforms client-side ee.Image.getRegion array to pandas.DataFrame
    df = pd.DataFrame(arr)

    # Rearrange the header.
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)

    # Remove rows without data inside.
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()

    # Convert the data to numeric values.
    for band in list_of_bands:
        df[band] = pd.to_numeric(df[band], errors='coerce')

    # Convert the time field into a datetime.
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')

    # Keep the columns of interest.
    df = df[['datetime',  *list_of_bands]]

    return df

def retrieve_GEE_data_as_pandas(year_list, era_lon, era_lat):

    import ee
    import pandas as pd
    import numpy as np

    var_list = ['minimum_2m_air_temperature',
                'maximum_2m_air_temperature',
                'mean_2m_air_temperature',
                'total_precipitation', 
                # 'surface_solar_radiation_downwards_sum', 
                # 'surface_thermal_radiation_downwards_sum', 
                'dewpoint_2m_temperature', 
                'surface_pressure',
                'u_component_of_wind_10m',
                'v_component_of_wind_10m']

    var_list_land = ['surface_solar_radiation_downwards_sum', 
                'surface_thermal_radiation_downwards_sum', ]

    # Initialize the library.
    ee.Initialize()

    # Import the ERA5_LAND collection.
    era5 = ee.ImageCollection("ECMWF/ERA5/DAILY")
    # Import the ERA5_LAND collection.
    era5_land = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_RAW")

    era_poi = ee.Geometry.Point([era_lon, era_lat])

    # era5_df = pd.DataFrame(columns=var_list)

    for i0, year0 in enumerate(year_list):

        # Initial date of interest (inclusive).
        i_date = str(year0)+"-01-01"

        # Final date of interest (exclusive).
        f_date = str(year0+1)+"-01-01"

        geeid1 = era5.select(var_list).filterDate(i_date, f_date)
        geeid2 = era5_land.select(var_list_land).filterDate(i_date, f_date)

        era5_data = geeid1.getRegion(era_poi, 1000).getInfo()

        era5_land_data = geeid2.getRegion(era_poi, 1000).getInfo()

        # print(len(era5_data))

        era5_land_dftmp = ee_array_to_df(era5_land_data, var_list_land).set_index('datetime')

        era5_dftmp = ee_array_to_df(era5_data, var_list).set_index('datetime').join(era5_land_dftmp)

        print(year0)

        if i0 == 0:
            era5_df = era5_dftmp.copy()
        else:
            era5_df = pd.concat([era5_df, era5_dftmp], ignore_index=False)

    return era5_df    

In [ ]:
def format_forz(era5df):

    tidy_df = era5df.copy()

    # Silence copy warning
    pd.set_option('mode.chained_assignment', None)

    tidy_df['TMN'] = tidy_df['minimum_2m_air_temperature'] - 273.15
    tidy_df['TMX'] = tidy_df['maximum_2m_air_temperature'] - 273.15

    # Shortwave
    # To W
    tidy_df['RSDS'] = \
        tidy_df['surface_solar_radiation_downwards_sum']/3600/24
    # Remove negative noise
    tidy_df['RSDS'][tidy_df['RSDS'] < 0] = 0
    
    # RH
    TD = tidy_df['dewpoint_2m_temperature'] - 273.15
    T = tidy_df['mean_2m_air_temperature'] - 273.15
    tidy_df['RH'] = 100 * (np.exp((17.67 * TD) / (243.5 + TD)) /
                           np.exp((17.67 * T) / (243.5 + T)))
    # Force bounds
    tidy_df['RH'][tidy_df['RH'] > 100] = 100
    tidy_df['RH'][tidy_df['RH'] < 0] = 0

    e_VPa = 611.2 * np.exp((17.67 * TD) / (243.5 + TD))
    tidy_df['SH'] = 0.622 * e_VPa / (tidy_df['surface_pressure']*0.01 - 0.378 * e_VPa)

    tidy_df['TD'] = tidy_df['dewpoint_2m_temperature'] - 273.15

    tidy_df['VPA'] = e_VPa * 0.1

    # Wind
    U = tidy_df['u_component_of_wind_10m']
    V = tidy_df['v_component_of_wind_10m']
    tidy_df['WSPD'] = np.sqrt(U**2 + V**2)

    # Precipitation [m/day > mm/day]
    tidy_df['PRECC'] = \
        tidy_df['total_precipitation']*1000
    tidy_df['PRECC'][tidy_df['PRECC'] < 0] = 0

    tidy_df['Year'] = pd.to_datetime(tidy_df.index).year
    tidy_df['Day'] = pd.to_datetime(tidy_df.index).day_of_year
    tidy_df['Hour'] = 9
        
    # remove old columns
    del tidy_df['surface_solar_radiation_downwards_sum']
    del tidy_df['surface_thermal_radiation_downwards_sum']
    del tidy_df['dewpoint_2m_temperature']
    del tidy_df['u_component_of_wind_10m']
    del tidy_df['v_component_of_wind_10m']
    del tidy_df['total_precipitation']
    del tidy_df['surface_pressure']
    del tidy_df['minimum_2m_air_temperature']
    del tidy_df['maximum_2m_air_temperature']
    del tidy_df['mean_2m_air_temperature']

    return tidy_df[['Year','Day','Hour','TMX','TMN','RSDS','WSPD','RH','PRECC']]


In [ ]:
def write_clmt_file(era5_df_forz, outfile = '../examples/test_daily/clmt_1980.txt'):
    
	fid = open(outfile, 'wt')
	fid.write('DJ0306XDHMNRWHP'+'\n')
	fid.write('CCWSRM'+'\n')
	fid.write('{:0.2f},{:0.2f},{:0.2f}'.format(10,1,13)+'\n')
	fid.write('{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f}'.format(7,0,0,0,0,0,0,0,0,0,0,0,0)+'\n')
	for i in era5_df_forz.index:
		fid.write('{:0.1f},{:0.1f},{:0.1f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.2f},{:0.8f}'.format(era5_df_forz['Year'][i],
										era5_df_forz['Day'][i],
										era5_df_forz['Hour'][i],
										era5_df_forz['TMX'][i],
										era5_df_forz['TMN'][i],
										era5_df_forz['RSDS'][i],
										era5_df_forz['WSPD'][i],
										era5_df_forz['RH'][i],
										era5_df_forz['PRECC'][i]) + '\n')
	fid.close()

In [ ]:
year_list = np.arange(1979, 2019)

sitename = 'Barrow'

site_lat = {'Altay':47.733, 'Kabahe':47.883, 'Fuyun':47.200, 'Ebo': 38.00, 
            'Barrow':71.309033, 'DrewPoint':70.904, 'Yakutsk':62.02, 
            'Wudaoliang':35.218, 'PT8':38.67, 'PT1': 38.7822,
            'Fairbanks':64.80, 'OldMan': 66.45, 'HappyValley':69.1466}
site_lon = {'Altay':88.083, 'Kabahe':86.200, 'Fuyun':89.783, 'Ebo': 100.92, 
            'Barrow':-156.661517, 'DrewPoint':-153.634, 'Yakutsk': 129.72, 
            'Wudaoliang':93.0833, 'PT8': 98.96, 'PT1':98.7452,
            'Fairbanks':-147.77, 'OldMan': -150.6167, 'HappyValley':-148.85}

# Define the locations of interest in degrees.
era_lon = site_lon[sitename]
era_lat = site_lat[sitename]

print('lat:',era_lat, 'lon:', era_lon)

In [ ]:
update_data = True

if update_data:

    out = retrieve_GEE_data_as_pandas(year_list, era_lon, era_lat)

    out.to_csv('raw_from_GEE.csv')

In [ ]:
raw = pd.read_csv('raw_from_GEE.csv', index_col=0, date_format="%Y-%m-%d")

raw.index

# pd.to_datetime(raw.index).year

In [ ]:
for i0, year0 in enumerate(year_list):
    
	data0 = raw[raw.index.year == year0]

	data1 = format_forz(data0)

	write_clmt_file(data1, outfile = '../examples/test_daily/clmt_'+str(year0)+'.txt')

In [ ]:
data365 = regroup_365days(format_forz(raw))

write_clmt_file(data365, outfile = '../examples/test_daily/clmt_spinup.txt')

In [ ]:
data365.TMN.plot()
data365.TMX.plot()

In [ ]:
data365.RSDS.plot()

In [ ]:
data365.PRECC.plot()